In [24]:
# df = m13.data.read_df()
# m1.sorted_data.read_df()
m14.data.read_df()

,date,instrument,pe_ttm_0,price_limit_status_0
0,2021-01-04,000001.SZA,13.382400,2.0
1,2021-01-05,000001.SZA,13.073022,2.0
2,2021-01-06,000001.SZA,14.073104,2.0
3,2021-01-07,000001.SZA,14.317728,2.0
4,2021-01-08,000001.SZA,14.281754,2.0
...,...,...,...,...
390974,2021-05-26,688036.SHA,44.940617,2.0
390975,2021-05-27,688036.SHA,44.948277,2.0
390976,2021-05-28,688036.SHA,41.621140,2.0
390977,2021-05-31,688036.SHA,45.451305,2.0


In [23]:
# df.info()
df.describe().T 

,count,mean,std,min,25%,50%,75%,max
pe_ttm_0,214074.0,3.174231e+01,2.418404e+02,-1.236946e+04,1.118912e+01,2.073719e+01,3.829444e+01,6.785655e+03
m:open,214074.0,3.012336e+02,8.142983e+02,2.066906e+00,1.776812e+01,8.329990e+01,2.590475e+02,1.405279e+04
m:high,214074.0,3.064117e+02,8.262258e+02,2.127401e+00,1.805740e+01,8.482786e+01,2.638065e+02,1.411358e+04
m:close,214074.0,3.015462e+02,8.151627e+02,2.087071e+00,1.778547e+01,8.330581e+01,2.592585e+02,1.392700e+04
m:low,214074.0,2.964887e+02,8.034647e+02,2.056824e+00,1.752255e+01,8.192073e+01,2.546698e+02,1.364202e+04
m:amount,214074.0,5.977655e+08,9.472490e+08,4.279149e+06,1.282167e+08,2.882107e+08,6.653356e+08,2.490733e+10
label,214074.0,8.344516e+00,2.804756e+00,0.000000e+00,7.000000e+00,8.000000e+00,1.000000e+01,1.900000e+01


In [18]:
# 本代码由可视化策略环境自动生成 2021年12月10日 13:23
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# 回测引擎：初始化函数，只执行一次
def m4_initialize_bigquant_run(context):
    # 加载预测数据
    context.ranker_prediction = context.options['data'].read_df()

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    # 预测数据，通过options传入进来，使用 read_df 函数，加载到内存 (DataFrame)
    # 设置买入的股票数量，这里买入预测股票列表排名靠前的5只
    stock_count = 5
    # 每只的股票的权重，如下的权重分配会使得靠前的股票分配多一点的资金，[0.339160, 0.213986, 0.169580, ..]
    context.stock_weights = T.norm([1 / math.log(i + 2) for i in range(0, stock_count)])
    # 设置每只股票占用的最大资金比例
    context.max_cash_per_instrument = 0.6
    context.options['hold_days'] = 5

    from zipline.finance.slippage import SlippageModel
    class FixedPriceSlippage(SlippageModel):
        def process_order(self, data, order, bar_volume=0, trigger_check_price=0):
            if order.limit is None:
                price_field = self._price_field_buy if order.amount > 0 else self._price_field_sell
                price = data.current(order.asset, price_field)
            else:
                price = data.current(order.asset, self._price_field_buy)
            # 返回希望成交的价格和数量
            return (price, order.amount)
    # 设置price_field,默认是开盘买入，收盘卖出
    context.fix_slippage = FixedPriceSlippage(price_field_buy='open', price_field_sell='close')
    context.set_slippage(us_equities=context.fix_slippage)
# 回测引擎：每日数据处理函数，每天执行一次
def m4_handle_data_bigquant_run(context, data):
    # 获取当前持仓
    positions = {e.symbol: p.amount * p.last_sale_price
                 for e, p in context.portfolio.positions.items()}
    print(f'****** \n \n 获取当前持仓:{positions}')

    today = data.current_dt.strftime('%Y-%m-%d')
    # 按日期过滤得到今日的预测数据
    ranker_prediction = context.ranker_prediction[context.ranker_prediction.date == today]

    # 大盘风控模块，读取风控数据
    benckmark_risk = ranker_prediction['bm_0'].values[0]

    # 当risk为1时，市场有风险，全部平仓，不再执行其它操作
    if benckmark_risk > 0:
        for instrument in positions.keys():
            context.order_target(context.symbol(instrument), 0)
        print(today, '大盘风控止损触发,全仓卖出')
        return

    # 1. 资金分配
    # 平均持仓时间是hold_days，每日都将买入股票，每日预期使用 1/hold_days 的资金
    # 实际操作中，会存在一定的买入误差，所以在前hold_days天，等量使用资金；之后，尽量使用剩余资金（这里设置最多用等量的1.5倍）
    # print(f'交易日计数{context.trading_day_index}，账户信息：{context.portfolio}')
    is_staging = context.trading_day_index < context.options['hold_days']  # 是否在建仓期间（前 hold_days 天）
    cash_avg = context.portfolio.portfolio_value / context.options['hold_days']  # 持仓期平均价值
    cash_for_buy = min(context.portfolio.cash, (1 if is_staging else 1.5) * cash_avg)  #
    cash_for_sell = cash_avg - (context.portfolio.cash - cash_for_buy)  # 持仓应卖资金
    print(f'买入资金预算{cash_avg}, 实际可用现金{cash_for_buy}，卖出后可得到得到现金{cash_for_sell}')

    # 2. 根据需要加入移动止赢止损模块、固定天数卖出模块
    stock_sold = []  # 记录卖出的股票，防止多次卖出出现空单

    # ------------------------START:止赢止损模块(含建仓期)---------------
    current_stopwin_stock = []
    current_stoploss_stock = []
    positions_cost = {e.symbol: p.cost_basis for e, p in context.portfolio.positions.items()}
    if len(positions) > 0:
        for instrument in positions.keys():
            stock_cost = positions_cost[instrument]
            stock_market_price = data.current(context.symbol(instrument), 'price')
            # 赚9%且为可交易状态就止盈
            if stock_market_price / stock_cost - 1 >= 0.09 and data.can_trade(context.symbol(instrument)):
                context.order_target_percent(context.symbol(instrument), 0)
                cash_for_sell -= positions[instrument]
                current_stopwin_stock.append(instrument)
            # 亏5%并且为可交易状态就止损
            if stock_market_price / stock_cost - 1 <= -0.05 and data.can_trade(context.symbol(instrument)):
                context.order_target_percent(context.symbol(instrument), 0)
                cash_for_sell -= positions[instrument]
                current_stoploss_stock.append(instrument)
        if len(current_stopwin_stock) > 0:
            print(today, '止盈股票列表', current_stopwin_stock)
            stock_sold += current_stopwin_stock
        if len(current_stoploss_stock) > 0:
            print(today, '止损股票列表', current_stoploss_stock)
            stock_sold += current_stoploss_stock
    # --------------------------END: 止赢止损模块--------------------------

    # --------------------------START：持有固定天数卖出(不含建仓期)-----------
    current_stopdays_stock = []
    positions_lastdate = {e.symbol: p.last_sale_date for e, p in context.portfolio.positions.items()}
    # 不是建仓期（在前hold_days属于建仓期）
    if not is_staging:
        for instrument in positions.keys():
            # 如果上面的止盈止损已经卖出过了，就不要重复卖出以防止产生空单
            if instrument in stock_sold:
                continue
            # 今天和上次交易的时间相隔hold_days就全部卖出 datetime.timedelta(context.options['hold_days'])也可以换成自己需要的天数，比如datetime.timedelta(5)
            if data.current_dt - positions_lastdate[instrument] >= datetime.timedelta(5) and data.can_trade(context.symbol(instrument)):
                context.order_target_percent(context.symbol(instrument), 0)
                current_stopdays_stock.append(instrument)
                cash_for_sell -= positions[instrument]
        if len(current_stopdays_stock) > 0:
            print(today, '固定天数卖出列表', current_stopdays_stock)
            stock_sold += current_stopdays_stock
    # -------------------------  END:持有固定天数卖出-----------------------

    # -------------------------- START: ST和退市股卖出 ---------------------
    st_stock_list = []
    for instrument in positions.keys():
        try:
            instrument_name = ranker_prediction[ranker_prediction.instrument == instrument].name.values[0]
            # 如果股票状态变为了st或者退市 则卖出
            if 'ST' in instrument_name or '退' in instrument_name:
                if instrument in stock_sold:
                    continue
                if data.can_trade(context.symbol(instrument)):
                    context.order_target(context.symbol(instrument), 0)
                    st_stock_list.append(instrument)
                    cash_for_sell -= positions[instrument]
        except:
            continue
    if st_stock_list != []:
        print(today, '持仓出现st股/退市股', st_stock_list, '进行卖出处理')
        stock_sold += st_stock_list

    # -------------------------- END: ST和退市股卖出 ---------------------

    # 3. 生成轮仓卖出订单：hold_days天之后才开始卖出；对持仓的股票，按机器学习算法预测的排序末位淘汰
    if not is_staging and cash_for_sell > 0:
        instruments = list(reversed(list(ranker_prediction.instrument[ranker_prediction.instrument.apply(lambda x: x in positions)])))  # 反向排序
        for instrument in instruments:
            # 如果资金够了就不卖出了
            if cash_for_sell <= 0:
                break
            # 防止多个止损条件同时满足，出现多次卖出产生空单
            if instrument in stock_sold:
                continue
            context.order_target(context.symbol(instrument), 0)
            cash_for_sell -= positions[instrument]
            stock_sold.append(instrument)

    # 4. 生成轮仓买入订单：按机器学习算法预测的排序，买入前面的stock_count只股票
    # 计算今日跌停的股票
    dt_list = list(ranker_prediction[ranker_prediction.price_limit_status_0 == 1].instrument)  # 股价在收盘时的涨跌停状态：1表示跌停，2表示未涨跌停，3则表示涨停
    # 计算今日ST/退市的股票
    st_list = list(ranker_prediction[ranker_prediction.name.str.contains('ST') | ranker_prediction.name.str.contains('退')].instrument)
    # 计算所有禁止买入的股票池
    banned_list = stock_sold + dt_list + st_list
    buy_cash_weights = context.stock_weights
    buy_instruments = [k for k in list(ranker_prediction.instrument) if k not in banned_list][:len(buy_cash_weights)]
    max_cash_per_instrument = context.portfolio.portfolio_value * context.max_cash_per_instrument
    for i, instrument in enumerate(buy_instruments):
        cash = cash_for_buy * buy_cash_weights[i]
        if cash > max_cash_per_instrument - positions.get(instrument, 0):  # 减去已经持有的仓位
            # 确保股票持仓量不会超过每次股票最大的占用资金量
            cash = max_cash_per_instrument - positions.get(instrument, 0)
        if cash > 0:  # 如果需要增持
            context.order_value(context.symbol(instrument), cash)



# 回测引擎：准备数据，只执行一次
def m4_prepare_bigquant_run(context):
    pass
def m4_before_trading_start_bigquant_run(context, data):
    # 获取涨跌停状态数据
    df_price_limit_status = context.ranker_prediction.set_index('date')
    today=data.current_dt.strftime('%Y-%m-%d')
    # 得到当前未完成订单
    for orders in get_open_orders().values():
        # 循环，撤销订单
        for _order in orders:
            ins=str(_order.sid.symbol)
            try:
                #判断一下如果当日涨停，则取消卖单
                if  df_price_limit_status[df_price_limit_status.instrument==ins].price_limit_status_0.ix[today]>2 and _order.amount<0:
                    cancel_order(_order)
                    print(today,'尾盘涨停取消卖单',ins) 
            except:
                continue

m1 = M.instruments.v2(
    start_date='2018-01-01',
    end_date='2021-01-01',
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m2 = M.advanced_auto_labeler.v2(
    instruments=m1.data,
    label_expr="""# #号开始的表示注释
# 0. 每行一个，顺序执行，从第二个开始，可以使用label字段
# 1. 可用数据字段见 https://bigquant.com/docs/data_history_data.html
#   添加benchmark_前缀，可使用对应的benchmark数据
# 2. 可用操作符和函数见 `表达式引擎 <https://bigquant.com/docs/big_expr.html>`_

# 计算收益：5日收盘价(作为卖出价格)除以明日开盘价(作为买入价格)
shift(close, -5) / shift(open, -1)

# 极值处理：用1%和99%分位的值做clip
clip(label, all_quantile(label, 0.01), all_quantile(label, 0.99))

# 将分数映射到分类，这里使用20个分类
all_wbins(label, 20)

# 过滤掉一字涨停的情况 (设置label为NaN，在后续处理和训练中会忽略NaN的label)
where(shift(high, -1) == shift(low, -1), NaN, label)
""",
    start_date='',
    end_date='',
    benchmark='000300.SHA',
    drop_na_label=True,
    cast_label_int=True
)

m3 = M.input_features.v1(
    features="""# #号开始的表示注释
# 多个特征，每行一个，可以包含基础特征和衍生特征
#return_5
#return_10
#return_20
#avg_amount_0/avg_amount_5
#avg_amount_5/avg_amount_20
#rank_avg_amount_0/rank_avg_amount_5
#rank_avg_amount_5/rank_avg_amount_10
#rank_return_0
#rank_return_5
#rank_return_10
#rank_return_0/rank_return_5
#rank_return_5/rank_return_10
pe_ttm_0
"""
)

m15 = M.general_feature_extractor.v7(
    instruments=m1.data,
    features=m3.data,
    start_date='',
    end_date='',
    before_start_days=0
)

m16 = M.derived_feature_extractor.v3(
    input_data=m15.data,
    features=m3.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False
)

m7 = M.join.v3(
    data1=m2.data,
    data2=m16.data,
    on='date,instrument',
    how='inner',
    sort=False
)

m21 = M.chinaa_stock_filter.v1(
    input_data=m7.data,
    index_constituent_cond=['沪深300'],
    board_cond=['全部'],
    industry_cond=['全部'],
    st_cond=['正常'],
    delist_cond=['非退市'],
    output_left_data=False
)

m13 = M.dropnan.v1(
    input_data=m21.data
)

m5 = M.stock_ranker_train.v6(
    training_ds=m13.data,
    features=m3.data,
    test_ds=m13.data,
    learning_algorithm='排序',
    number_of_leaves=30,
    minimum_docs_per_leaf=1000,
    number_of_trees=20,
    learning_rate=0.1,
    max_bins=1023,
    feature_fraction=1,
    data_row_fraction=1,
    plot_charts=True,
    ndcg_discount_base=1,
    m_lazy_run=False
)

m6 = M.input_features.v1(
    features_ds=m3.data,
    features="""# #号开始的表示注释
# 多个特征，每行一个，可以包含基础特征和衍生特征
price_limit_status_0"""
)

m9 = M.instruments.v2(
    start_date=T.live_run_param('trading_date', '2021-01-01'),
    end_date=T.live_run_param('trading_date', '2021-06-01'),
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m17 = M.general_feature_extractor.v7(
    instruments=m9.data,
    features=m6.data,
    start_date='',
    end_date='',
    before_start_days=0
)

m18 = M.derived_feature_extractor.v3(
    input_data=m17.data,
    features=m6.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False
)

m23 = M.chinaa_stock_filter.v1(
    input_data=m18.data,
    index_constituent_cond=['沪深300'],
    board_cond=['全部'],
    industry_cond=['全部'],
    st_cond=['正常'],
    delist_cond=['非退市'],
    output_left_data=False
)

m14 = M.dropnan.v1(
    input_data=m23.data
)

m8 = M.stock_ranker_predict.v5(
    model=m5.model,
    data=m14.data,
    m_lazy_run=False
)

m10 = M.select_columns.v3(
    input_ds=m14.data,
    columns='date,instrument,price_limit_status_0',
    reverse_select=False
)

m11 = M.join.v3(
    data1=m8.predictions,
    data2=m10.data,
    on='date,instrument',
    how='left',
    sort=True
)

m20 = M.input_features.v1(
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
name"""
)

m22 = M.use_datasource.v1(
    instruments=m9.data,
    features=m20.data,
    datasource_id='instruments_CN_STOCK_A',
    start_date='',
    end_date=''
)

m24 = M.input_features.v1(
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
bm_0 = where(close/shift(close,5)-1<-0.05,1,0)"""
)

m12 = M.index_feature_extract.v3(
    input_1=m9.data,
    input_2=m24.data,
    before_days=100,
    index='000300.HIX'
)

m26 = M.select_columns.v3(
    input_ds=m12.data_1,
    columns='date,bm_0',
    reverse_select=False
)

m25 = M.join.v3(
    data1=m22.data,
    data2=m26.data,
    on='date',
    how='left',
    sort=True
)

m27 = M.join.v3(
    data1=m11.data,
    data2=m25.data,
    on='date,instrument',
    how='left',
    sort=True
)

m19 = M.sort.v4(
    input_ds=m27.data,
    sort_by='date,position',
    group_by='--',
    keep_columns='--',
    ascending=True
)

m4 = M.trade.v4(
    instruments=m9.data,
    options_data=m19.sorted_data,
    start_date='',
    end_date='',
    initialize=m4_initialize_bigquant_run,
    handle_data=m4_handle_data_bigquant_run,
    prepare=m4_prepare_bigquant_run,
    before_trading_start=m4_before_trading_start_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='close',
    capital_base=1000001,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark=''
)


[2021-12-10 12:02:39.508300] INFO: moduleinvoker: instruments.v2 开始运行..

[2021-12-10 12:02:39.543534] INFO: moduleinvoker: 命中缓存

[2021-12-10 12:02:39.546748] INFO: moduleinvoker: instruments.v2 运行完成[0.039869s].

[2021-12-10 12:02:39.557896] INFO: moduleinvoker: advanced_auto_labeler.v2 开始运行..

[2021-12-10 12:02:39.570970] INFO: moduleinvoker: 命中缓存

...

[2021-12-10 12:05:53.760322] INFO: Performance: Simulated 98 trading days out of 98.

[2021-12-10 12:05:53.762632] INFO: Performance: first open: 2021-01-04 09:30:00+00:00

[2021-12-10 12:05:53.764069] INFO: Performance: last close: 2021-06-01 15:00:00+00:00

[2021-12-10 12:06:19.616583] INFO: moduleinvoker: backtest.v8 运行完成[35.011462s].

[2021-12-10 12:06:19.618687] INFO: moduleinvoker: trade.v4 运行完成[35.18414s].